In [29]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from dotenv import load_dotenv
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [30]:
load_dotenv()

True

In [31]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation"
)
model = ChatHuggingFace(llm=llm)

In [32]:
class ChatState(TypedDict):

    messages: Annotated[list[BaseMessage], add_messages]

In [33]:
def chat_node(state: ChatState):

    
    messages = state['messages']

   
    response = model.invoke(messages)

    
    return {'messages': [response]}

In [34]:
checkpointer=MemorySaver()
graph = StateGraph(ChatState)


graph.add_node('chat_node', chat_node)

graph.add_edge(START, 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [35]:
#initial_state = {
#    'messages': [HumanMessage(content='What is the capital of india')]
#}

#chatbot.invoke(initial_state)['messages'][-1].content

In [36]:
thread_id='1'
while True:

    user_message=input('Type_here: ')

    print("User:", user_message)

    if user_message.strip().lower() in ['quit','bye', 'exit']:
        break
    
    config={'configurable':{'thread_id': thread_id}}
    response= chatbot.invoke({'messages': [HumanMessage(content=user_message)]}, config=config)

    print('AI: ', response['messages'][-1].content)


User: hi, my name is bhola
AI:   Hello Bhola! It's nice to meet you. How can I assist you today? If you have any questions or need help with something, feel free to ask. I'm here to help!
User: what's my name
AI:   Your name is Bhola, as you mentioned earlier. How can I assist you today, Bhola? If you have any questions or need help with something, feel free to ask. I'm here to help!
User: quit


In [40]:
chatbot.get_state(config=config)

StateSnapshot(values={'messages': [HumanMessage(content='hi, my name is bhola', additional_kwargs={}, response_metadata={}, id='46d5b662-a32f-460a-a2c7-a6f6042fad62'), AIMessage(content=" Hello Bhola! It's nice to meet you. How can I assist you today? If you have any questions or need help with something, feel free to ask. I'm here to help!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 12, 'total_tokens': 55}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.3', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--c299ec2e-da16-4bd5-8f9e-7781c8fc25ab-0', usage_metadata={'input_tokens': 12, 'output_tokens': 43, 'total_tokens': 55}), HumanMessage(content="what's my name", additional_kwargs={}, response_metadata={}, id='80d9b002-fa9d-4e05-9008-de89c56b2adf'), AIMessage(content=" Your name is Bhola, as you mentioned earlier. How can I assist you today, Bhola? If you have any questions or need help with someth